# 1. Erstellen eines Datensatzes und Annotieren der Bilder

Um einen Objekterkennungs-Algorithmus zu trainieren braucht es, wie für alle Deep Learning Anwendungen, Daten. Aus diesen Daten wird das Modell später versuchen, die Merkmale herauszuziehen, die für das Erkennen des jeweiligen Objektes relevant sind. 

![Image](../computerVisionProjekt/beispielbilder/grafiken/object-detection-example.PNG)
**Objekterkennung mit Bounding-Boxen** *- das Modell erkennt verschiedene Objekte, wie Autos, Menschen, Hydranten, etc. und identifiziert sie im Bild, indem eine Box um das jeweilige Objekt gezogen wird. Die Zahl gibt jeweils an, wie sicher sich das Modell ist, dass es sich beim identifizierten Objekt tatsächlich um ein solches handelt.*

Um das Verständnis zu vertiefen und um auf wichtige Aspekte aufmerksam zu machen haben wir eine Reihe von Fragen und Aufgaben in dieses Notebook eingebaut:

**Beispiel**

1. **Verständnisfragen**

<div class="alert alert-block alert-info">
<b>Frage:</b> In den blauen Boxen stehen Verständnisfragen</div>

2. **Arbeitsaufträge**

<div class="alert alert-block alert-success">
<b>Auftrag:</b> In grünen Boxen finden sich Arbeitsaufträge.</div>

Besonders komplizierte Aspekte oder Bereiche, in denen leicht Fehler gemacht werden können, sind mit einer roten Box gekennzeichnet:

<div class="alert alert-block alert-danger">
<b>Achtung:</b> Hier müssen Sie gut aufpassen. Nehmen Sie sich ausreichend Zeit, die nachfolgenden Angaben zu lesen oder Fragen Sie ein Mitglied des Teams um Hilfe.</div>

**Was *ist* Objekterkennung?**

Da es sich hierbei um ein Problem des **überwachten Lernens** handelt, reicht es nicht einfach, dem Algorithmus nur die Bilddaten zuzuführen. Damit eine Zuordnung von Bild &rarr; Objekt stattfinden kann, müssen die Daten entsprechend **annotiert** sein. 

<details>
    <summary><b>Exkurs: Algorithmendesign vs. Datenkuration</b></summary>
    <h5>Wo liegen heute die Constraints?</h5>

In vielen Bereichen des Deep Learning spielt das Design der Algorithmen heute nicht mehr die Hauptrolle. Insbesondere in der maschinellen              Bildverarbeitung, also <b>Computer Vision</b> sind die <i>besten</i> (oder zumindest: sehr gute) Algorithmen heute bekannt. Der Constraint für die     Performance der Modelle liegt also heute weniger im Bereich der Architektur, sondern mehr in der Qualität der Datensätze. 
    
    
</details>
<br>
Das Annotieren von Daten kann teilweise automatisiert werden, wird aber in den meisten Fällen noch von Menschen per Hand durchgeführt. 

In diesem Notebook widmen wir uns also drei Dingen:

1. **Dem Sammeln von Bilddaten**
2. **Der Datenannotation**
3. **Finetuning eines SOTA-Objekterkenners**

### 1.1. Wo bekommt man die Daten her?

Je nach Problemstellung kann es leichter oder schwieriger sein, an (gute) Daten heranzukommen. Für einige Domänen existieren bereits große - teilweise auch gut aufbereitete - Datensätze, die oft auch frei zugänglich im Internet abrufbar sind. 

Beispiele sind:

* ImageNet
* NIST-Datensätze
* MS COCO
* u.v.m.

Die Klassen des **MS COCO** - Datensatzes. **COCO** steht dabei für **C**ommon **O**bjects in **CO**ntext.

![Image](../computerVisionProjekt/beispielbilder/grafiken/coco.JPG)

Hier einige Beispielbilder mit **Bounding-Boxen**:

![Image](../computerVisionProjekt/beispielbilder/grafiken/coco_examples.JPG)

<a href="https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8839032&tag=1">Quelle</a>

Eine kleine Übersicht über populäre Datensätze findet sich <a href="https://www.kaggle.com/datasets">hier</a>.
Es existieren auch viele weitere Repositories, über die Daten verhältnismäßg einfach bezogen werden können. 

Viele Problemstellungen sind jedoch sehr spezifisch für das Unternehmen und den konkreten Anwendungskontext. Hier ist es deutlich weniger wahrscheinlich, dass bereits gute Datensätze existieren oder dass diese - falls es sie gibt - frei verfügbar sind.

In diesem Fall müssen die Daten erst selbst gesammelt und annotiert werden. Das kann - je nach Problem - einen **großen bis sehr großen Aufwand bedeuten**. 

Wir werden uns im Folgenden selbst einen Datensatz erstellen und diesen annotieren.

## 1.2. Der Kontext

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Ihr Unternehmen stellt eine Reihe an Werkzeugen und anderen Bauteilen her. Einige dieser Objekte werden über das selbe Förderband weiter zum Lager transportiert. Aktuell müssen Menschen die Objekte per Hand in die korrekten Aufbewahrungsbehältnisse sortieren. 

Sie möchten diesen Prozess automatisieren und hatten die Idee, künstliche Intelligenz dafür zu verwenden.</div>

## 1.3. Importieren der notwendigen Bibliotheken

Um die Bilder aufzunehmen und diese zu rendern, verwenden wir die Computer-Vision Bibliothek <a href="https://opencv.org/"><b>OpenCV</b></a>. Ein tieferes Verständnis der Bibliothek wird nicht vorausgesetzt.

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Führen Sie die nächste Codezelle aus um die benötigten Bibliotheken zu importieren.</div>

In [1]:
import cv2                                                         # OpenCV
import os                                                          # Betriebssystem
from ultralytics import YOLO                                       # Machine Learning Bibliothek
from IPython.display import Video                                  # Zum Anzeigen von Videodateien

# Die folgenden Funktionen dienen der Unterstützung beim Erstellen der Daten und der Verwendung des Modells
###########################################################################################################
from helpers.image_helpers import capture_images             # Helferfunktion zum Aufnehmen der Bilder
from helpers.model_helpers import inference_webcam # Helferfunktion zum Anwenden des Modells mit der Webcam
from helpers.model_helpers import inference_video            # Helferfunktion zum Anwenden des Modells mit einer Videodatei
from helpers.model_helpers import choose_model               # Helferfunktion zur Auswahl des Modells
from helpers.model_helpers import set_training_config        # Helferfunktion zum Einstellen der Trainingskonfiguration
from helpers.model_helpers import get_trained_model          # Helferfunktion zum Extrahieren eines trainierten Modells
from helpers.file_handlers import prepare_folder_structure   # Helferfunktion zum Erstellen der benötigten Ordnerstruktur
from helpers.file_handlers import create_config              # Helferfunktion zum Erstellen der Konfigurationsdatei
from helpers.file_handlers import move_file                  # Helferfunktion zum Speichern des Modells
from helpers.config import set_metadata                      # Helferfunktion zum Erstellen des Ordners
from helpers.config import set_image_data                    # Helferfunktion zum Erstellen des Ordners
from helpers.file_handlers import cleanup_images

TASK = 'DETECT'


## 1.4 Aufnehmen der Bilder



<div class="alert alert-block alert-success">
<b>Auftrag:</b> Führen Sie die nächste Codezelle aus und legen Sie einen Gruppennamen fest.</div>

In [2]:
GROUP, PATH = set_metadata(TASK)


Gruppenordner erstellt: ..\data\detection\deneme


<div class="alert alert-block alert-danger">
<b>Achtung:</b> Hier wurde der Ordner erstellt und anschließend in den nächsten Schritten automatisch alle Bilder dorthin verschoben
: /computerVisionProjekt/data/detection/[Ihre_Gruppenname]</div>

<div class="alert alert-block alert-success">
<b>Auftrag:</b>

<ol start="1">
<li> Öffnen Sie nun die Kamera-App 
<li> Machen Sie die gewünschten Fotos (mintestens: 50 Bilder)
<li> Löschen sie die Fotos die sie nicht behalten möchten

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Führen Sie die nächste Codezelle aus damit Ihre Bilder automatisch in den entsprechenden Ordner verschoben werden..</div>


In [ ]:
from pathlib import Path
from helpers.move_recent_images import move_recent_images

if __name__ == "__main__":
    source = Path.home() / "Pictures" / "Camera Roll"
    target = Path.home() / "computervision" / "data" / "detection"
 
    # Optional: sicherstellen, dass Zielverzeichnis existiert
    target.mkdir(parents=True, exist_ok=True)

    move_recent_images(source, target, hours=2)


⚠️ Kein Unterordner in detection gefunden.


<div class="alert alert-block alert-danger">
<b>Auftrag:</b>Dieses Skript funktioniert möglicherweise nicht auf jedem Computer korrekt, da die Pfade und Zugriffsrechte variieren können. Im Falle eines Ausführungsfehlers lesen Sie bitte die Dokumentation zu Ausführungsfehlern im Moodle-Kurs.</div>


# 2. Annotieren mit Label-Studio

Zum Annotieren der Bilder verwenden wir <a href="https://labelstud.io/">Label-Studio</a>. Die Auswahl der Annotations-Software spielt in diesem Fall eine untergeordnete Rolle. Wir könnten genauso gut eine andere Software verwenden (z.b. <a href="https://www.cvat.ai/">CVAT</a>). 

Sollten Sie bereits eine Annotationssoftware kennen, verwenden Sie am besten die, mit der Sie am vertrautesten sind. Wichtig ist lediglich, dass sich über die Software die Daten im `YOLO`-Format exportieren lassen. 

<div class="alert alert-block alert-danger">
<b>Achtung:</b>Die Anleitung zum Starten von Label Studio finden Sie im Dokument „cv-technik-anleitung“ im Moodle-Kurs.</div>

### 2.1. Eine kurze Tour durch Label-Studio

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Machen Sie sich mit der Funktionsweise von Label Studio vertraut und annotieren Sie die Bilder. Dafür können Sie dem untenstehenden Guide folgen, der Ihnen eine kurze Einführung in das Programm gibt. Oder fragen Sie uns, wir erklären Ihnen gerne die Funktionsweise von Label-Studio.</div>

**1. Home-Screen: Projektübersicht**

   Nach dem Öffnen landen Sie in der Projektübersicht. 

   <div class="alert alert-block alert-success">
<b>Auftrag:</b> Erstellen Sie Ihre Projekt <b>Gruppen Name</b> aus.</div>
   
![Image](../computerVisionProjekt/beispielbilder/guides/labelstudio-guide/ls_guide1.png)

**2. Projektansicht**

   Die Projektansicht ist noch leer. Das liegt daran, dass noch keine Daten in das Projekt geladen wurden.
   Dies kann man über den **Import**-Button tun.

   <div class="alert alert-block alert-success">
<b>Auftrag:</b> Laden Sie die aufgenommenen Bilder in Label-Studio. Dazu wählen Sie <b>upload files</b> aus und navigieren dort zum Projektordner.</div>


![Image](../computerVisionProjekt/beispielbilder/guides/labelstudio-guide/ls_guide2.png)

   Die Bilder erscheinen dann in der Projektübersicht.

![Image](../computerVisionProjekt/beispielbilder/guides/labelstudio-guide/ls_guide3.png)

**3. Bestimmen der Label: Settings**

   Bevor mit dem Annotieren begonnen werden kann, müssen noch die Label festgelegt werden. Dies kann über die **Settings** (oben rechts) tun.

**4. Settings: Labeling Interface**

   Navigieren Sie zum Punkt **Labeling Interface**.

![Image](../computerVisionProjekt/beispielbilder/guides/labelstudio-guide/ls_guide4.png)



**5. Labeling Interface: **Browse templates****

Wählen Sie **Object detection with bounding boxes**. Hier können Sie im Feld **Add label names** jetzt Ihre Klassen hinzufügen. Dafür geben Sie einfach für jede Klasse den Namen in das Feld und fügen diese mit **Add** den Labels hinzu. Sie können auch mehrere Klassen auf einmal hinzufügen. Dafür muss für jede Klasse eine eigene Zeile angefangen werden. Nach Bedarf kann auch die Farbe der Labels verändert werden. 

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Fügen Sie die gewünschten Label hinzu.</div>

<div class="alert alert-block alert-danger">
<b>Achtung:</b> Nachdem alle Label hinzugefügt wurden, muss das mit <b>Save</b> bestätigt werden!</div>

![Image](../computerVisionProjekt/beispielbilder/guides/labelstudio-guide/ls_guide5.png)


**6. Annotieren: Bounding Boxes**

   Die Annotations-Ansicht besteht zum einen aus dem aktuellen Bild, den Labels / Klassen (unten) und zwei weiteren Bereichen (rechts). Letztere sind für den weiteren Verlauf nicht von Bedeutung.

<div class="alert alert-block alert-danger">
<b>Achtung:</b> Pro Bild können <b>beliebig</b> viele Objekte annotiert werden. Es sollten natürlich möglichst alle Vorkommnisse eines Objekts annotiert werden.!</div>

<div class="alert alert-block alert-danger">
<b>Hinweise:</b>
1.Je enger und genauer (waagerecht oder senkrecht) die Labelbox am Gegenstand ausgerichtet ist, desto besser wird das Ergebnis.
2.beachten sie das eine Trennung zwischen Gegenstand und Background gibt</div>
   
   Es gibt verschiedene Möglichkeiten, Bilder zu annotieren. In diesem Projekt verwenden wir die **Bounding Box**. Dabei handelt es sich um eine rechteckige Box, die über das jeweilige Objekt gezogen wird. Im Bild unten sind die Objekte beispielsweise Bilderrahmen, Pflanzen, Sofas und Tische.

   ![Image](../computerVisionProjekt/beispielbilder/guides/labelstudio-guide/ls_guide7.png)

Um ein Objekt einer Klasse zu annotieren **muss** vorher das entsprechende Label unten ausgewählt werden (hier kann auch der Shortcut (die entsprechende Zahl, die jeweils rechts im Label steht) verwendet werden). **Anschließend** wird mit der Maus eine **rechteckige Box** um das Objekt gezogen. Die Box sollte so eng wie möglich um das Objekt gezogen werden.

   Wenn alle Objekte auf dem Bild annotiert wurden, können die Annotationen mit **Submit** gespeichert werden - das nächste Bild öffnet sich dann automatisch. Unerwünschte Bilder können mit **Skip** übersprungen werden.

   Das wird so lange gemacht, bis alle Bilder annotiert wurden.

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Annotieren Sie die Bilder in Ihrem Projekt.</div>

**7. Projektübersicht: Label All Tasks**

   Navigieren Sie zurück in die Übersicht (Projects/Lernumgebung). Jetzt können Sie anfangen, die Bilder zu annotieren.

   <div class="alert alert-block alert-success">
<b>Auftrag:</b> Öffnen Sie die Annotier-Ansicht. Klicken Sie dafür auf <b>Label All Tasks</b></div>

![Image](../computerVisionProjekt/beispielbilder/guides/labelstudio-guide/ls_guide6.png)



**8. Projektübersicht: Exportieren**

   Navigieren Sie nun zurück in die Übersicht und exportieren Sie die Bilder mit den Annotationen im **YOLO-Format**.

   <div class="alert alert-block alert-success">
<b>Auftrag:</b> Exportieren Sie die Bilder im YOLO-Format. Dazu wählen Sie zunächst <b>Export</b> aus und anschließend das <b>YOLO</b>-Format. Dann bestätigen Sie das ganze mit <b>Export</b>. Jetzt werden die Daten automatisch als ZIP-Datei heruntergeladen. </div>

   

![Image](../computerVisionProjekt/beispielbilder/guides/labelstudio-guide/ls_guide8.png)

# 3. Vorbereiten der Bilddaten für das Modelltraining

Bevor mit dem Modelltraining begonnen werden kann, müssen die annotierten Daten noch in die entsprechenden Ordner entzippt werden.

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Entpacken Sie den soeben heruntergeladenen ZIP-Ordner und verschieben Sie alle darin enthaltenen Dateien wie train und val, in die entsprechenden Ordner , die Sie zu Beginn erstellt haben.“<code><dynamischer_Basispfad>/computervision/data/detection/IHR_GRUPPENNAME</code>-Ordner.</div>

Im Ordner befinden sich mehrere Dinge:

>* images
>* labels
>* classes.txt
>* notes.json

Im `images`-Ordner befinden sich die Bilder und im `labels`-Ordner die dazugehörigen Labels. Das YOLO-Modell erwartet, dass die Bilder und Label nocheinmal aufgeteilt werden in ein **Trainingsset** und ein **Validationsset**. 

Sie brauchen also diese Ordnerstruktur:

* images
    * train
    * val
* labels
    * train
    * val
* classes.txt
* notes.json

Die ```prepare_folder_structure(PATH)```-Funktion stellt diese Ordnerstruktur automatisch für Sie her und teilt die Bilder und Label in **Trainings-** und **Validationsdaten** ein.

Anschließend wird über die `create_config(PATH)`-Funktion die Konfigurationsdatei erstellt, die dem Modell sagt, wo die Daten zu finden sind.

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Führen Sie die nachfolgende Codezelle mit einem Klick auf den Play-Button aus.</div>

In [ ]:
prepare_folder_structure(PATH, task=TASK)
create_config(PATH, task=TASK)
cleanup_images(PATH)


Ihr `IHR_GRUPPENNAME`-Order sollte nun so aussehen:

![Image](../computerVisionProjekt/beispielbilder/grafiken/ordner-detect.PNG)

Außerdem sollte sich eine `config.yaml` - Datei im Ordner befinden.

# 4. Auswahl eines geeigneten Modells und Fine-Tuning für eigene Daten

Eine der erfolgreichsten Objekterkennungs-Architekturen, ist die YOLO-Reihe. YOLO steht für:

* **Y**ou
* **O**nly
* **L**ook
* **O**nce

Die YOLO-Modellarchitektur ermöglicht die schnelle und effiziente Erkennung von Objekten in Bildern und Videos. Im Gegensatz zu anderen Methoden benötigt YOLO nur einen Durchgang, um alle Objekte zu identifizieren. Es berücksichtigt unterschiedliche Objektgrößen und bietet Vielseitigkeit für verschiedene Anwendungen. YOLO zeichnet sich durch seine Geschwindigkeit und Fähigkeit aus, komplexe Szenen in einem Schritt zu erfassen.

Wenn Sie mehr über die YOLO-Modelle lesen möchten, finden Sie <a href="https://blog.roboflow.com/guide-to-yolo-models/">hier</a> einen guten Blog-Artikel und <a href="https://docs.ultralytics.com/">hier</a> die Ultralytics-Dokumentation zum YOLOv8-Modell.

<div class="alert block" style="background-color: #D3D3D3;">
<b>Info:</b> Kürzlich wurde eine neue Version des Modells veröffentlicht: YOLOv10!</div>

**Schematischer Aufbau** (keine Angst - wir werden in diesem Notebook nichts berechnen!)

![Image](../computerVisionProjekt/beispielbilder/grafiken/yolo_functionality.JPG)

## 4.1. Auswahl des Modells

Um eines der Modelle zu laden, erstellen muss einfach eine Instanz der `YOLO()`-Klasse erstellt werden. Dabei geben Sie die Modellarchitektur an, die Sie haben möchten.

Es gibt verschiedene Varianten der YOLO-Modelle, die jeweils für andere **Tasks** gedacht sind (Objekterkennung, Klassifizierung, Bildsegmentierung, Posen/Punkterkennung)

Für die Objekterkennung gibt es das Modell in verschiedenen Größen:

![Image](../computerVisionProjekt/beispielbilder/grafiken/yolo.png)

Je größer das Modell ist, desto besser - aber auch **teurer** (Rechenleistung) und **langsamer** (Rechenzeit) - wird das Modell.

<div class="alert alert-block alert-info">
<b>Frage:</b> Überlegen Sie, in welchen Situationen welche Modellgröße verwendet werden sollte.</div>

Die Syntax um ein Modell zu laden, lautet wie folgt:

```Python

modell = YOLO('modellversion')
```

Gibt man einen ganzen Pfad an und zuletzt die gewünscht Modellversion, wird das Modell automatisch an den Ort im Pfad heruntergeladen. Wir haben eine Helferfunktion geschrieben, die Ihnen den gewünschten Pfad zum Modell automatisch generiert.

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Führen Sie die nächste Codezelle aus, um ein <code>YOLO</code>-Modell zu laden. Wählen Sie die Modellnummer aus, um das gewünschte Modell zu laden. Zum Beispiel,tragen Sie 1 bis 4 da oben nach ausfühurng von code aus.</div>

In [ ]:
model =choose_model(TASK)
#1:'yolov11n (Nano)',
#2: 'yolov11s (Small)',
#3: 'yolov11m (Medium)',
#4: 'yolov11l (Large)'

## 4.2. Verwenden des Modells

Nachdem das Modell geladen wurde, kann es direkt verwendet werden - **aber warum?** 

Die `.pt` - Datei (**P**y**T**orch) enthält ein **vortrainiertes Modell**. Dieses wurde auf dem COCO128 - Subset (**C**ommon **O**bjects in **CO**ntext) trainiert und ist in der Lage, eine große Anzahl von häufig vorkommenden Objekten zu erkennen.

### 4.2.1 Teste gerne das vortrainierte Modell mit der Webcam

Das vortrainierte Modell kennt Ihre Bilder noch nicht. Testen Sie gerne das Modell aus und beobachten Sie, wie dieses auf Ihre Gegenstände reagiert.

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Führen Sie die nächste Codezelle aus, um das vortrainierte Modell mit der Webcam zu verwenden.</div>
<div class="alert alert-block alert-danger">
<b>Achtung:</b> Sobald Sie den Code der nachsten Zelle ausführen aktiviert sich die eingebaute Webcam. Sie können dieses Fenster schließen indem Sie auf die Taste <b>q</b> drücken. </div> 

In [1]:
inference_webcam(model=model,
                 device=0,
                 verbose=False) # Verbose bestimmt, ob der Modell-Output im Notebook angezeigt wird.

NameError: name 'inference_webcam' is not defined

## 4.3. Finetuning des Modells auf den eigenen Bildern

Nachdem Sie gesehen haben, wie man das Modell grundsätzlich verwenden kann, sollten Sie es nun auf Ihre eigenen Bedürfnisse abstimmen.

Wenn die Aufgabe nicht gerade ist, Dinge zu erkennen, die im COCO-Datensatz enthalten sind (Personen, Tische, Handys, ...), müssen Sie dem Modell zwangsweise noch beibringen, wie die gewünschten Objekte aussehen.

Hier haben Sie zwei Möglichkeiten:

1. Sie trainieren ein Modell von Grund auf neu.
2. Sie nehmen ein vortrainiertes Modell und trainieren es mit den eigenen Bildern weiter.

Der zweite Schritt wird als **Fine-Tuning** bezeichnet und macht sich die Technik des **Transfer-Learnings** zu nutze.

Beim Transfer-Learning nutzen wir die Tatsache aus, dass das vorherige Modell bereits viele allgemeine Merkmale erlernt hat, die auch für unsere spezifische Aufgabe relevant sein könnten. 

Der Vorteil von Transfer-Learning liegt darin, dass man von den bereits erworbenen Fähigkeiten des Modells profitieren und somit oft mit weniger Daten und Rechenzeit auskommen kann. Der vortrainierte Teil des Modells erkennt grundlegende Merkmale wie Kanten, Formen und Textur, während der neu trainierte Teil auf die spezifischen Objekte oder Muster in unseren eigenen Daten fein abgestimmt wird.

**Transfer-Learning ist besonders nützlich in Situationen, in denen es schwierig oder kostspielig ist, einen umfangreichen eigenen Datensatz zu sammeln.** Es ermöglicht uns, auf den Schultern bereits existierender Modelle zu stehen und diese für unsere individuellen Anforderungen anzupassen.

## 4.4. Das Training

Wenn alles vorbereitet ist, kann das Modell mit einem einfachen Befehl trainiert werden:

```Python
results = model.train(data='config.yaml', epochs=10)
```

Dazu müssen der `.train()`-Funktion zwei Argumente übergeben werden: 
* die `config.yaml`-Datei
* die **Anzahl der Trainingsepochen** - eine Epoche ist ein gesamter Durchlauf **aller** Daten durch das Modell.

### 4.4.1. Die Trainingsfunktion

Um das Modell zu trainieren müssen Sie diesem nur mitteilen, wo die Trainingsdaten zu finden sind. Dafür haben Sie vorher die Konfigurationsdatei erzeugt. 

```Python
results = model.train(data=DATA_PATH,   # Zeigt auf den Ordner mit der Konfigurationsdatei
                      epochs=EPOCHS,    # 1 Epoche = ein Durchlauf aller Daten durch das Modell
                      name=NAME,        # Name der einzelnen Trainingsdurchläufe
                      project=SAVE_DIR) # Name des Ordners in dem die Modellergebnisse gespeichert werden     
```

Beim Ausführen der nachfolgenden Codezelle werden Sie dazu aufgefordert die Anzahl der **Trainingsepochen** einzugeben. Wie lange das Modell trainieren soll ist immer problemabhängig, aber für ein leichtes Problem sollten 30-50 Epochen vollkommen ausreichend sein. 

<div class="alert alert-block alert-success"> 
<b>Auftrag:</b> Führen Sie die nächste Codezelle aus, um das Modell zu trainieren. Verändern Sie gerne die Anzahl der Epochen. Beobachten Sie dabei die Ausgabe - während des Trainings werden verschiedene Metriken getrackt.
</div>

Grundsätzlich wollen wir, dass der **Modellfehler** (*loss*) im Verlauf des Trainings sinkt, und die **Genauigkeit** (*precision*) steigt. 

Beim Training des YOLO-Modells werden verschiedene Losses (**box_loss, cls_loss, dfl_loss**) und drei Precision-Metriken mitverfolgt (**P, mAP50, mAP50-95**). Was genau sich hinter den Metriken versteckt, können Sie in den nachfolgenden Details nachlesen.

<details>
<summary>Was bedeuten die Metriken?</summary>

* **Box Loss (Bounding Box Loss):**
Die Bounding Box Loss ist eine Kennzahl, die die Genauigkeit der Vorhersagen für begrenzende Kästen (BoundingBoxes) misst. Es bewertet, wie gut das Modell die Position und Größe eines erkannten Objekts vorhersagt, indem es die Differenz zwischen den vorhergesagten und den tatsächlichen begrenzenden Kastenwerten quantifiziert.

* **Cls Loss (Classification Loss):**
Die Klassifikationsverlustfunktion misst die Genauigkeit der vorhergesagten Klassen. In der Objekterkennung geht es darum, nicht nur die Position der Objekte zu lokalisieren (BoundingBox), sondern auch die Objekte selbst zu identifizieren. Die Klassifikationsverlustfunktion bewertet, wie gut das Modell die richtige Klasse für ein erkanntes Objekt vorhersagt.

* **Dfl Loss (Directional Field Loss):**
Der Directional Field Loss ist relevant für Aufgaben, bei denen die Orientierung oder Ausrichtung eines erkannten Objekts wichtig ist, wie zum Beispiel bei der Erkennung von Fahrzeugen. Diese Verlustfunktion bewertet die Genauigkeit der vorhergesagten Orientierung des erkannten Objekts im Vergleich zur tatsächlichen Orientierung.

* **mAP50 (mean Average Precision at 50% IoU):**
mAP steht für "mean Average Precision". Es ist eine Metrik, die die Qualität von Objekterkennungsmodellen bewertet. Der Wert 50% IoU (Intersection over Union) gibt an, wie gut die vorhergesagten BoundingBoxen mit den tatsächlichen BoundingBoxen übereinstimmen müssen, damit eine Vorhersage als korrekt betrachtet wird. mAP50 berechnet den durchschnittlichen Präzisionswert über verschiedene Klassen bei einem IoU-Schwellenwert von 50%.

* **mAP95 (mean Average Precision at 95% IoU):**
Ähnlich wie mAP50, jedoch mit einem höheren IoU-Schwellenwert von 95%. Dies bedeutet, dass die BoundingBoxen viel genauer mit den tatsächlichen BoundingBoxen übereinstimmen müssen, um als korrekt betrachtet zu werden. mAP95 gibt eine strengere Bewertung der Modellleistung.

Außerdem werden Precision und Recall angezeigt:

* **Precision / P (Genauigkeit):**
Precision misst, wie viele der als positiv vorhergesagten Instanzen tatsächlich positiv sind. Es wird durch die Formel

$\text{Precision} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}$

berechnet. Hohe Precision bedeutet, dass die vorhergesagten positiven Instanzen tendenziell korrekt sind, jedoch sagt es nichts darüber aus, wie viele tatsächlich positive Instanzen das Modell verpasst hat (False Negatives).

* **Recall / R (Sensitivität oder Trefferquote):**
Recall misst, wie viele der tatsächlich positiven Instanzen vom Modell erkannt wurden. Es wird durch die Formel

$\text{Recall} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}$

berechnet. Hoher Recall bedeutet, dass das Modell eine hohe Anzahl der tatsächlich positiven Instanzen findet, aber es sagt nichts darüber aus, wie viele der vorhergesagten positiven Instanzen tatsächlich korrekt sind (True Positives im Verhältnis zu False Positives).

Diese beiden Metriken sind oft im Konflikt zueinander. Eine Erhöhung der Precision kann zu einer Verringerung des Recalls führen und umgekehrt. Das F1-Score ist eine Metrik, die Precision und Recall kombiniert und häufig verwendet wird, um ein ausgewogenes Maß für die Leistung eines Modells zu erhalten. Der F1-Score wird durch die Formel

$\text{F1} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$

berechnet.

</details>

<div class="alert alert-block alert-info"> 
<b>Frage:</b> Wie entwickeln sich der Modellfehler und die Genauigkeit im Verlaufe des Trainings? Ist die Entwicklung wünschenswert?
</div>

In [ ]:
EPOCHS, DATA_PATH, NAME, SAVE_DIR = set_training_config(PATH)

results = model.train(data=DATA_PATH, 
                      epochs=EPOCHS, 
                      name=NAME,
                      project=SAVE_DIR)

### 4.4.2. Was ist beim Training passiert?

Nachdem das Modell trainiert wurde, wurde im Ordner `GRUPPENNAME/runs/` automatisch ein Ordner für den Trainingsdurchlauf angelegt. Die Inhalte dieses Ordners sehen folgendermaßen aus:

![Image](../computerVisionProjekt/beispielbilder/grafiken/eval-detect.PNG)

Im `weights`-Ordner befinden sich zwei Dateien: `best.pt` und `last.pt`:

* `best.pt` ist der beste Zustand des Modells
* `last.pt` ist der letzte Zustand des Modells

Je nachdem ob Sie weitertrainieren möchten oder das Modell in die Produktionsumgebung überführen wollen, können Sie mit einer der beiden Dateien weiterarbeiten. 

<div class="alert alert-block alert-info">
<b>Frage:</b> Warum macht es im Hinblick auf Überanpassung Sinn, immer den besten Zustand des Modells zu speichern?</div>

Außerdem enthält der Ordner noch eine Reihe an anderen Dateien:

* Konfusionmatrizen
* results.png
* results.csv
* Trainings- und Validationsbatches
* ...

Die Datei `results.png` zeigt beispielsweise eine Übersicht über verschiedene Metriken:

![Image](../computerVisionProjekt/beispielbilder/grafiken/results.png)

<div class="alert alert-block alert-success">
<b>Frage:</b> Nehmen Sie sich Zeit diese Dateien in Ruhe zu betrachten. Welche Informationen können Sie herauslesen?</div>

### 4.4.3. Das trainierte Modell verwenden 

Zu Beginn haben Sie ein Modell mithilfe des Befehls:

```Python

model = YOLO('pfad/zum/model.pt')
```
geladen.

Dasselbe können Sie mit diesen Dateien auch machen!

<div class="alert alert-block alert-success">
<b>Auftrag</b>: Führen Sie die nächste Codezelle aus, um das trainierte Modell zu laden.</div>

In [ ]:
trained_model = get_trained_model(PATH) # extrahiert das zuletzt trainierte Modell,
                                        # wenn ein bestimmter run ausgewählt werden soll, kann das mit dem 'run' Parameter gemacht werden,
                                        # z.B. get_trained_model(PATH, run=2)
trained_model = YOLO(trained_model)

Somit haben Sie in der Variable `trained_model` die beste Version des akutellen Trainingsdurchlaufs gespeichert. 
Dieses Modell können Sie jetzt genauso verwenden, wie Sie die Basis-Version des Modells zu Beginn verwendet haben:

<div class="alert alert-block alert-success"> 
<b>Auftrag</b>: Führen Sie die nächste Codezelle aus und testen Sie nun ob Ihr Modell die Gegenstände erkennt wenn Sie diese vor die Webcam halten. </div>

In [ ]:
inference_webcam(model=trained_model,
                 device=0,
                 verbose=False)

## 4.5. Evaluieren des Modells

Evaluieren Sie die Leistung des trainierten Modells und überprüfen, ob es bereits die gewünschten Ergebnisse erzielt. Falls nicht, sollten Sie mögliche Gründe analysieren und Ansatzpunkte identifizieren die zur Verbesserung des Modells beitragen könnten. Betrachten Sie insbesondere folgende Faktoren:

**Trainingszeit:** Wurde das Modell ausreichend lange trainiert? Eine längere Trainingsdauer könnte zu einer besseren Leistung führen.

**Datenqualität:** Wie ist die Qualität der Trainingsdaten? Sind sie ausreichend hochwertig und decken sie verschiedene Szenarien ab? Haben Sie ausreichend Daten?

*Weitere Hyperparameter:* Die betrachten wir nicht direkt - das würde zu weit führen. Behalten Sie aber im Hinterkopf, dass auch hier Veränderungen vorgenommen werden könnten.

<div class="alert alert-block alert-success"> 
<b>Aufgabe:</b> Reflektieren Sie über potenzielle Gründe, warum das Modell in bestimmten Situationen nicht die erwarteten Ergebnisse liefert. Entwickeln Sie Strategien zur Lösung dieser Probleme und zur weiteren Verbesserung der Modellleistung.
</div>

<details>
<summary>Tipps</summary>
    <h3>Tips zur Verbesserung des trainierten YOLO-Objekterkennungsmodells:</h2>
    <ul>
        <li><strong>Visualisiert eure Vorhersagen:</strong> Nutzt Visualisierungen, um die Vorhersagen des Modells auf neuen Bildern zu überprüfen. Schaut euch an, wo das Modell Fehler macht, und versucht, Muster zu erkennen.</li>
        <li><strong>Achtet auf fehlende Klassen:</strong> Kontrolliert, ob das Modell bestimmte Objektklassen nicht erkennt. Vielleicht müsst ihr mehr Trainingsdaten für diese Klassen sammeln oder die Gewichtung der Klassen anpassen.</li>
        <li><strong>Überprüft die Klassenverteilung:</strong> Stellt sicher, dass die Anzahl der Trainingsbilder für jede Klasse ungefähr gleich ist. Falls nicht, kann das zu Ungleichgewichten führen.</li>
        <li><strong>Passt die Trainingszeit an:</strong> Wenn nötig, erhöht die Anzahl der Trainingsepochen, um sicherzustellen, dass das Modell gut konvergiert und optimale Gewichtungen erreicht.</li>
        <li><strong>Kontrolliert die Datenqualität:</strong> Schaut euch die Qualität der Trainingsdaten an. Stellt sicher, dass die Beschriftungen korrekt sind, minimiert Rauschen und sorgt für eine Vielfalt in den Daten.</li>
        <li><strong>Berücksichtigt die Objektgrößen:</strong> Denkt darüber nach, ob das Modell für die Größe der erkannten Objekte optimal eingestellt ist. Das Hinzufügen von Daten mit unterschiedlichen Objektgrößen könnte hilfreich sein.</li>
        <li><strong>Experimentiert mit Augmentierung:</strong> Probiert verschiedene Augmentierungstechniken aus, um die Datenvielfalt zu erhöhen und das Modell robuster gegenüber verschiedenen Bedingungen zu machen.</li>
        <li><strong>Nutzt Transfer Learning:</strong> Falls möglich, startet das Training mit einem bereits auf großen Datensätzen trainierten YOLO-Modell und feintuned es auf eure spezifische Aufgabe.</li>
        <li><strong>Achtet auf Overfitting:</strong> Kontrolliert, ob das Modell möglicherweise zu sehr auf die Trainingsdaten angepasst ist (Overfitting). Reguliert dies durch Anpassungen von Regularisierungs- und Dropout-Techniken.</li>
        <li><strong>Beschriftungen überprüfen:</strong> Stellt sicher, dass die Beschriftungen (Annotationen) korrekt sind und jedes Objekt in den Bildern genau markiert ist. Fehlerhafte Beschriftungen können zu Verwirrungen und schlechter Leistung führen.</li>
        <li><strong>Verschiedene Aufnahmeszenarien einbeziehen:</strong> Berücksichtigt verschiedene Beleuchtungsverhältnisse, Wetterbedingungen und Hintergründe, um sicherzustellen, dass das Modell in verschiedenen Umgebungen gut funktioniert.</li>
        <li><strong>Objektvariationen erfassen:</strong> Achtet darauf, dass verschiedene Variationen der Objekte eurer Klassen in den Daten enthalten sind. Das Modell sollte in der Lage sein, Objekte unabhängig von ihrer Position, Orientierung oder Größe zu erkennen.</li>
        <li><strong>Berücksichtigung von Schatten und Reflexionen:</strong> Wenn möglich, nehmt Bilder auf, die Schatten oder Reflexionen der Objekte enthalten. Dies stellt sicher, dass das Modell auch unter realen Bedingungen robust arbeitet.</li>
        <li><strong>Mehr Daten sammeln:</strong> Je mehr Daten ihr habt, desto besser. Sammelt weitere Bilder, um die Vielfalt eurer Daten zu erhöhen und dem Modell mehr Beispiele für jede Klasse zu bieten.</li>
        <li><strong>Korrektes Labeling von schwierigen Fällen:</strong> Achtet besonders auf schwierige Fälle, in denen Objekte teilweise verdeckt oder schwer zu erkennen sind. Stellt sicher, dass diese Fälle korrekt beschriftet sind, um das Modell in schwierigen Situationen zu verbessern.</li>
        <li><strong>Bewegte Objekte erfassen:</strong> Wenn möglich, fangt auch Bilder von sich bewegenden Objekten ein. Dies ist besonders relevant, wenn eure Anwendung bewegte Objekte enthält.</li>
        <li><strong>Verschiedene Blickwinkel einbeziehen:</strong> Variiert die Aufnahmeperspektiven, um sicherzustellen, dass das Modell Objekte aus verschiedenen Blickwinkeln erkennen kann.</li>
        <li><strong>Kontinuierliche Verbesserung:</strong> Überprüft eure Daten regelmäßig und verbessert die Beschriftungen oder fügt neue Daten hinzu, um die Qualität im Laufe der Zeit zu steigern.</li>
    </ul>
</details>

## 4.6. Re-Training

Wenn nötig, trainieren Sie das Modell neu!

**Training**
<div class="alert alert-block alert-success"> 
<b>Auftrag:</b> Führen Sie die nächste Codezelle aus, um das Modell neu zu trainieren.
</div>

In [ ]:
model = choose_model(task='DETECT')

EPOCHS = int(input('Anzahl Epochen: '))
DATA_PATH = os.path.join(PATH, 'config.yaml')
NAME = f'detect-{GROUP}'
SAVE_DIR = os.path.join(PATH,'runs')

results = model.train(data=DATA_PATH, 
                      epochs=EPOCHS, 
                      name=NAME,
                      project=SAVE_DIR)

Testen Sie das neu trainierte Modell! Achten Sie darauf, das Modell aus dem richtigen Trainingsdurchlauf zu verwenden!

In [ ]:
latest_model = os.path.join(PATH, 'runs', f'{NAME}', 'weights', 'best.pt')
trained_model = YOLO(latest_model)

inference_webcam(model=trained_model,
                 device=0,
                 verbose=False)